In [1]:
#### Aim: To calculate background concentrations of CO2 and PM and to estimate Emission factor by Carbon Balance method.
## EF = ((Pa-Pb)/(CO2a-CO2b))*Wc
## Pa- Abolute Pollutant Concentration, Pb- Background Pollutant concentration, Wc- weight carbon fraction in fuel.

In [2]:
import pandas as pd
import numpy as np

### Uploading Dataset

In [3]:
data_PM=pd.read_excel('Dummy_Atmos_PM_Data.xlsx',na_values=['NaN'], parse_dates=['dt_time'])

data_gas=pd.read_excel('Dummy_GasSensor_Data.xlsx', sheet_name='Sensor_16', usecols= ['DateTime','CO2'],parse_dates=['DateTime'],na_values=['NaN'])

### Defining Time 

In [4]:
data_gas['minute'] = data_gas['DateTime'].dt.minute
data_gas['hour']=data_gas['DateTime'].dt.hour
data_gas['day']=data_gas['DateTime'].dt.day

data_PM['minute'] = data_PM['dt_time'].dt.minute
data_PM['hour']=data_PM['dt_time'].dt.hour
data_PM['day']=data_PM['dt_time'].dt.day

### Absolute and Background Dataframe- CO2.

Background Hourly mean-  Background hourly mean concentration (considering 1 percentile of last 10 seconds).

Absolute mean -  Hourly mean concentration.

In [5]:
total_hours=data_gas.groupby(['hour']).mean().index.get_level_values(0) 
df_percentile=pd.DataFrame([])
dataframe_Gas=pd.DataFrame([])
p=0;
for x in total_hours: # Available hours
    df_percentile=pd.DataFrame([])
    for y in range(len(data_gas)):
        if x == data_gas['hour'][y]: # Identifying single hour data from data_gas
            if y>9: # Starting from 10th datapoint/second
                
                #-----Forming Percentile DataFrame of CO2------
                df_10seconds=pd.DataFrame([data_gas['CO2'][y-10:y]]).T # Forming dataset of 10 seconds
                percentile_eachsecond=pd.DataFrame([np.nanpercentile(df_10seconds,1)]) # 1 perecntile value (of single timestep) from 10 second dataset.
                df_percentile=pd.concat([percentile_eachsecond,df_percentile]) # Concatenating each percentile value to get perecntile dataframe.
    
     # ----Calculation of Hourly Percentile  mean of CO2----
    each_hour_percentile_mean=pd.DataFrame([df_percentile.mean()],[x]) # Mean of hourly percentile dataframe, x= denotes hour 
    each_hour_percentile_mean.reset_index(drop=True, inplace=True) # Resetting index to concat horizontally in percentile_dataframe
    
    # -----Calculation of Absolute Hourly mean of CO2-------
    hourly_mean=data_gas.groupby(['day','hour'])['CO2'].mean() # Absolute Hourly mean of CO2.
    hour_single_mean=pd.DataFrame([hourly_mean.iloc[p]]) # Mean of single hour starting from first houri.e. p=0; 
    hour_single_mean.reset_index(drop=True,inplace=True) # Resetting index to concat horizontally in percentile_dataframe.
    p=p+1;
    
    # -----Combining Absolute and Percentile dataframe-------
    dataframe_initial=pd.concat([pd.DataFrame([x]),each_hour_percentile_mean,hour_single_mean], axis =1) # x=xth hour
    dataframe_Gas=pd.concat([dataframe_initial,dataframe_Gas],ignore_index=True)    
    
dataframe_Gas.columns=['Hours','Background_CO2','Absolute_CO2'] # defining columns of dataframe.

### Absolute and Background Dataframe- PM2.5, PM1 and PM10.

In [6]:
total_hours=data_PM.groupby(['hour']).mean().index.get_level_values(0)
p=0;
df_percentile=pd.DataFrame([])
dataframe_PM=pd.DataFrame([])
for x in total_hours: # Available hours
    df_percentile=pd.DataFrame([])
    for y in range(len(data_PM)):
        if x == data_PM['hour'][y]: # Identifying single hour data from data_PM
            if y>9:
                #-----Forming Percentile DataFrame of PM------
                df_PM25_10seconds=pd.DataFrame([data_PM['pm25raw'][y-10:y]]).T # PM2.5
                percentile_PM25=pd.DataFrame([np.nanpercentile(df_PM25_10seconds,1)])# 1 perecntile of last 10 seconds
                df_PM1_10seconds=pd.DataFrame([data_PM['pm1cnc'][y-10:y]]).T  # PM1
                percentile_PM1=pd.DataFrame([np.nanpercentile(df_PM1_10seconds,1)])
                df_PM10_10seconds=pd.DataFrame([data_PM['pm10cnc'][y-10:y]]).T  # PM10
                percentile_PM10=pd.DataFrame([np.nanpercentile(df_PM10_10seconds,1)])
                
                percentile=pd.concat([percentile_PM25,percentile_PM1,percentile_PM10], axis=1)
                df_percentile=pd.concat([percentile,df_percentile])
    
    # ----Calculation of Hourly Percentile  mean of PM----
    hourly_percentile_mean=df_percentile.mean(axis=0).reset_index().T # Percentile hourly mean dataframe.
    each_hour_percentile_mean=hourly_percentile_mean[1:]
   
    # -----Calculation of Absolute Hourly mean of PM-------
    hourly_mean_PM=data_PM.groupby(['day','hour'])['pm25raw','pm1cnc','pm10cnc'].mean() # Absolute Mean of all hours.
    hourly_mean_PM25=pd.DataFrame([hourly_mean_PM.iloc[p][0]]) # PM2.5
    hourly_mean_PM1=pd.DataFrame([hourly_mean_PM.iloc[p][1]]) # PM1
    hourly_mean_PM10=pd.DataFrame([hourly_mean_PM.iloc[p][2]]) # PM10
    
    # -----Combining Absolute and Percentile dataframe-------
    dataframe_initial_PM=pd.concat([pd.DataFrame([x]),each_hour_percentile_mean,hourly_mean_PM25,hourly_mean_PM1,hourly_mean_PM10], axis =1)
    dataframe_PM=pd.concat([dataframe_initial_PM,dataframe_PM], ignore_index=True)
    p=p+1;
    
dataframe_PM.columns=['Hours','Background_PM2.5','Background_PM1','Background_PM10','Absolute_PM2.5','Absolute_PM1','Absolute_PM10']

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_1588/1074433593.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hourly_mean_PM=data_PM.groupby(['day','hour'])['pm25raw','pm1cnc','pm10cnc'].mean() # Absolute Mean of all hours.


### Calculation of Emission Factor

In [7]:
Final_subset=pd.merge(dataframe_PM,dataframe_Gas)

Final_subset['CO2']=Final_subset['Absolute_CO2']-Final_subset['Background_CO2']
Final_subset['PM2.5']=Final_subset['Absolute_PM2.5']-Final_subset['Background_PM2.5']
Final_subset['PM1']=Final_subset['Absolute_PM1']-Final_subset['Background_PM1']
Final_subset['PM10']=Final_subset['Absolute_PM10']-Final_subset['Background_PM10']

# Estimating Ratio of PM against CO2
Final_subset['Ratio_PM2.5']=Final_subset['PM2.5']/Final_subset['CO2']
Final_subset['Ratio_PM1']=Final_subset['PM1']/Final_subset['CO2']
Final_subset['Ratio_PM10']=Final_subset['PM10']/Final_subset['CO2']

# Multiplying with carbon fraction
Final_subset['Emmission Factor_PM2.5']=0.86*Final_subset['Ratio_PM2.5'] # EF of PM2.5
Final_subset['Emmission Factor_PM1']=0.86*Final_subset['Ratio_PM1']   # EF of PM1
Final_subset['Emmission Factor_PM10']=0.86*Final_subset['Ratio_PM10']  # EF of PM10

Final_subset.to_excel('Dummy_EF.xlsx', index=True)